In [14]:
import json
import urllib3
import config
import pandas as pd
import string

http = urllib3.PoolManager()
call_string = 'www.thecocktaildb.com/api/json/' + config.api_version + '/' + config.api_key + '/search.php?f=' 

df_list = []
for i in string.ascii_lowercase:
    try :
        response = http.request('GET', call_string + i)
        data = json.loads(response.data)
        single_df = pd.json_normalize(data['drinks'])
        df_list.append(single_df)
    except:
        pass

df = pd.concat(df_list)
df.head(100)

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
0,17222,A1,None,None,None,Cocktail,None,Alcoholic,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",...,None,None,None,None,None,None,None,None,No,2017-09-07 21:42:09
1,13501,ABC,None,None,None,Shot,None,Alcoholic,Shot glass,Layered in a shot glass.,...,None,None,None,None,None,None,None,None,No,2016-08-31 19:32:08
2,17225,Ace,None,None,None,Cocktail,None,Alcoholic,Martini Glass,Shake all the ingredients in a cocktail shaker...,...,None,None,None,None,None,None,None,None,No,2017-09-07 22:05:06
3,14610,ACID,None,None,None,Shot,None,Alcoholic,Shot glass,Poor in the 151 first followed by the 101 serv...,...,None,None,None,None,None,None,None,None,No,2016-11-15 11:28:37
4,17837,Adam,None,"Alcoholic,Holiday",None,Ordinary Drink,None,Alcoholic,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",...,None,None,None,None,None,None,None,None,Yes,2016-09-02 11:29:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17005,A True Amaretto Sour,None,None,None,Cocktail,None,Alcoholic,Old-fashioned glass,Rub the rim of an old fashioned glass with lem...,...,None,None,None,None,None,None,None,None,No,2016-08-31 19:49:32
96,16134,Absolutly Screwed Up,None,None,None,Cocktail,None,Alcoholic,Collins glass,"Shake it up it tasts better that way, but you ...",...,None,None,None,None,None,None,None,None,No,2016-08-31 19:55:28
97,12710,Apple Berry Smoothie,None,None,None,Other / Unknown,None,Non alcoholic,Highball Glass,Throw everything into a blender and liquify.,...,None,None,None,None,None,None,None,None,No,2016-07-18 22:14:33
98,17229,Adios Amigos Cocktail,None,None,None,Cocktail,None,Alcoholic,Martini Glass,Shake together all the ingredients and strain ...,...,None,None,None,None,None,None,None,None,No,2017-09-07 22:31:33


In [15]:
df.to_excel('drinks.xlsx')
cols = df.columns.to_list()
ing_cols = []
meas_cols = []
rem_cols = []

for c in range(len(cols)):
    if 'strIngredient' in cols[c]:
        ing_cols.append(cols[c])
    if 'strMeasure' in cols[c]:
        meas_cols.append(cols[c])
    if 'strIngredient' not in cols[c] and 'strMeasure' not in cols[c]:
        rem_cols.append(cols[c])

ing_cols.append('idDrink')
meas_cols.append('idDrink')
print(ing_cols)
print(meas_cols)

['strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strIngredient10', 'strIngredient11', 'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15', 'idDrink']
['strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'idDrink']


In [16]:
df_ing = df[ing_cols].melt(id_vars = 'idDrink')
df_meas = df[meas_cols].melt(id_vars = 'idDrink')

# getting index to match ingredient to volume
df_ing['ing_id'] = df_ing['variable'].replace('(strIngredient)', '', regex=True)
df_meas['meas_id'] = df_meas['variable'].replace('(strMeasure)', '', regex=True)

# filtering out Nones
df_ing = df_ing[df_ing['value'].notnull()]
df_meas = df_meas[df_meas['value'].notnull()]

# ingredient merge
ing_meas = pd.merge(df_ing, df_meas, how = 'left', left_on = ['idDrink', 'ing_id'], right_on = ['idDrink', 'meas_id'])
ing_meas = ing_meas[['idDrink', 'ing_id', 'value_x', 'value_y']]

# final dataframe merge 
final_df = pd.merge(df[rem_cols], ing_meas, how = 'left', left_on = 'idDrink', right_on = 'idDrink')
final_df.sort_values('idDrink').head(50)

final_df.to_excel('drinks_rollout.xlsx')